In [127]:
from google.auth.transport.requests import Request
from google.oauth2.service_account import Credentials
import vertexai

key_path = "/home/ext_rufi_romang_gmail_com/serviceaccount.json"
credentials = Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"]
)

if credentials.expired:
    credentials.refresh(Request())

PROJECT_ID = "cfu-main"
REGION = "asia-southeast1"
vertexai.init(project=PROJECT_ID, location=REGION, credentials = credentials)

In [128]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

from langchain.chat_models import ChatVertexAI
from langchain.schema import AIMessage, HumanMessage, SystemMessage
llm = ChatVertexAI(model = "chat-bison",
                   max_output_tokens = 256,
                    temperature = 0.0,
                    top_p = 0.8,
                    top_k = 40,)

In [129]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain

In [130]:
prompt_1 = ChatPromptTemplate.from_template(
    "Berdasarkan chat, apakah dokter menyapa pasien sebelum sesi konsultasi? Apakah sapaan dokter menggunakan bahasa yang formal atau lebih friendly? Jika iya, tulis iya. Jika tidak, tulis tidak. Tidak perlu dijelaskan secara rinci"
    "\n\n{chat}")

chain_1 = LLMChain(llm=llm, prompt=prompt_1, output_key="Sapaan")

In [131]:
prompt_2 = ChatPromptTemplate.from_template(
    "Berdasarkan chat, apakah pasien memberikan keluhan? Jika iya, apakah dokter menanggapi keluhan tersebut?, Apakah dokter menanggapi keluhan tersebut? Jika iya, tulis iya. Jika tidak, tulis tidak. Tidak perlu dijelaskan secara rinci"
    "\n\n{chat}")

chain_2 = LLMChain(llm=llm, prompt=prompt_2, output_key="Respon_Dokter")

In [132]:
prompt_3 = ChatPromptTemplate.from_template(
    "Berdasarkan chat dan respon dokter, Apakah dokter memberikan diagnosis berdasarkan keluhan yang dijelaskan oleh pasien? Apakah dokter memberikan diagnosis berdasarkan keluhan yang dijelaskan oleh pasien? Jika iya, tulis iya. Jika tidak, tulis tidak. Tidak perlu dijelaskan secara rinci"
    "\n\chat: {chat} \n\nrespon dokter: {Respon_Dokter}")

chain_3 = LLMChain(llm=llm, prompt=prompt_3, output_key="Diagnosis")

In [133]:
prompt_4 = ChatPromptTemplate.from_template(
    "Berdasarkan chat dan respon dokter, apakah dokter memberikan saran kepada pasien? Jika iya, apakah saran yang diberikan berkaitan dengan keluhan yang dilakukan? Apakah dokter memberikan saran atau informasi tambahan lain? Jika iya, tulis iya. Jika tidak, tulis tidak. Tidak perlu dijelaskan secara rinci"
    "\n\chat: {chat} \n\nrespon dokter: {Respon_Dokter}")

chain_4 = LLMChain(llm=llm, prompt=prompt_4, output_key="Saran")

In [134]:
prompt_5 = ChatPromptTemplate.from_template(
    "Berdasarkan chat dan respon dokter, apakah dokter memberikan resep kepada pasien? Jika iya, tulis iya. Jika tidak, tulis tidak. Tidak perlu dijelaskan secara rinci"
    "\n\chat: {chat} \n\nrespon dokter: {Respon_Dokter}")

chain_5 = LLMChain(llm=llm, prompt=prompt_5, output_key="Resep")

In [135]:
prompt_6 = ChatPromptTemplate.from_template(
    "Berdasarkan sapaan, respon dokter, diagnosis, saran, dan resep, buatlah ringkasan mengenai performa dokter."
    "\n\sapaan: {Sapaan} \n\nrespon dokter: {Respon_Dokter} \n\n\ndiagnosis: {Diagnosis} \n\n\n\nsaran: {Saran} \n\n\n\n\nresep: {Resep}")

chain_6 = LLMChain(llm=llm, prompt=prompt_6, output_key="Performa")

In [136]:
prompt_7 = ChatPromptTemplate.from_template(
    "Berdasarkan 5 hal yg dinilai (output_key), berikan rating kepada dokter 1-10!"
    "\n\nSummary: {Performa}")

chain_7 = LLMChain(llm=llm, prompt=prompt_7, output_key="Rate")

In [137]:
chat1 = """
Dokter: klo bisa ditambah obat minumnya kka \
Pasien: obat apa ya dok? \
Dokter: maaf kk sdh menikah? \
Pasien: Sudah dok.. sedang menyusui \
Dokter: Usia anaknya? \
Pasien: Baru sebulan kurang seminggu dok \
Dokter: obat minumnya ctm 2x1 dimininum jika anaknya ada yang nunggu krn obat ini ngantuk \
Dokter: sama kk mandi PK y \
Pasien: Baik dok... berarti sama ctm ya dok \
Dokter: iy kka \
Pasien: Terimakasih dok.. ?? \
Dokter: hindarin makanan yg membuat kk alergi spt udang \
Dokter: sama 1 lg kka krn kulitnya kka sensitif, bedak sabun mandi body lotion semuanya pakai khusus /spt bayi \
Pasien: Kalau ceterizine sama ctm mending mana dong? Dirumah ada ceterizine. \
Dokter: Krn kk menyusui ctm aja ya \
Pasien: Ada rekomendasi ga dok bedak lotion dan sabun yang bagus? \
Pasien: Baik dok. Jadi ctm aja ya.. \ 
Dokter: Pokoknya khusus bayi \ 
Pasien: Baik dong... terima kasih dok.. ini sya pake salep ini berapa kalo pakai dok.? \
Dokter: Klo salep bebas mau dipakai berapa kali mis. habis mandi bisa disalepin \
Dokter: hnya pesan aja, jika mau disalepin dilap kering dulu \ 
Pasien: Baik dok. Mohon resep dokternya ya dok.. terima kasih banyak ya dokter.. ?? \
Dokter: jngan hbs disalepin ditmpa dgn salep lg, \
"""

In [138]:
chat2 = """
Dokter: mau obat ga?\
Pasien: obat apa ya dok? \
Dokter: obat nyamuk \
Pasien: gajelas dokter ya\
Dokter: kenapa \
Pasien: nawarin obat nyamuk \
Dokter: obat minumnya 2x sehari doang \
Dokter: sama harganya 25 ribu \
Pasien: gamau dok, saya sakit kepala \
Dokter: iy kka \
Pasien: Terimakasih dok.. ?? \
"""

In [139]:
overall_chain = SequentialChain(
    chains=[chain_1, chain_2, chain_3, chain_4, chain_5, chain_6, chain_7],
    input_variables = ["chat"],
    output_variables = ["Performa","Rate"],
    verbose=True)

In [140]:
consul1 = overall_chain(chat1)



> Entering new SequentialChain chain...

> Finished chain.


In [141]:
consul2 = overall_chain(chat2)



> Entering new SequentialChain chain...

> Finished chain.


In [142]:
embeddings = VertexAIEmbeddings("textembedding-gecko@003")

In [143]:
import json

In [144]:
with open("consul1.json", "w") as json_file:
    json.dump(consul1, json_file)

In [145]:
from langchain_community.document_loaders import JSONLoader

In [146]:
loader = JSONLoader(
    file_path = "/home/ext_ahmadpanduwiranata_gmail_com/Downloads/consul1.json",
    jq_schema='.',
    text_content=False,
    json_lines=True)

data = loader.load()

In [147]:
print(data[0])

page_content='{"chat": "\\nDokter: klo bisa ditambah obat minumnya kka Pasien: obat apa ya dok? Dokter: maaf kk sdh menikah? Pasien: Sudah dok.. sedang menyusui Dokter: Usia anaknya? Pasien: Baru sebulan kurang seminggu dok Dokter: obat minumnya ctm 2x1 dimininum jika anaknya ada yang nunggu krn obat ini ngantuk Dokter: sama kk mandi PK y Pasien: Baik dok... berarti sama ctm ya dok Dokter: iy kka Pasien: Terimakasih dok.. ?? Dokter: hindarin makanan yg membuat kk alergi spt udang Dokter: sama 1 lg kka krn kulitnya kka sensitif, bedak sabun mandi body lotion semuanya pakai khusus /spt bayi Pasien: Kalau ceterizine sama ctm mending mana dong? Dirumah ada ceterizine. Dokter: Krn kk menyusui ctm aja ya Pasien: Ada rekomendasi ga dok bedak lotion dan sabun yang bagus? Pasien: Baik dok. Jadi ctm aja ya.. \\\\ \\nDokter: Pokoknya khusus bayi \\\\ \\nPasien: Baik dong... terima kasih dok.. ini sya pake salep ini berapa kalo pakai dok.? Dokter: Klo salep bebas mau dipakai berapa kali mis. habis

In [148]:
#!rm -rf ./chroma
persist_directory = './chroma/'

In [149]:
from langchain.vectorstores import Chroma

In [150]:
vectordb = Chroma.from_documents(
    documents = data, 
    embedding = embeddings,
    persist_directory=persist_directory
)

In [151]:
vectordb.persist()

In [155]:
question = "Berapa rate dokter?"

In [156]:
hasil = vectordb.similarity_search(question, k=1)

In [163]:
for i in range (0, len(hasil)):
    print(hasil[i-1], "\n\n")

page_content='{"chat": "\\nDokter: klo bisa ditambah obat minumnya kka Pasien: obat apa ya dok? Dokter: maaf kk sdh menikah? Pasien: Sudah dok.. sedang menyusui Dokter: Usia anaknya? Pasien: Baru sebulan kurang seminggu dok Dokter: obat minumnya ctm 2x1 dimininum jika anaknya ada yang nunggu krn obat ini ngantuk Dokter: sama kk mandi PK y Pasien: Baik dok... berarti sama ctm ya dok Dokter: iy kka Pasien: Terimakasih dok.. ?? Dokter: hindarin makanan yg membuat kk alergi spt udang Dokter: sama 1 lg kka krn kulitnya kka sensitif, bedak sabun mandi body lotion semuanya pakai khusus /spt bayi Pasien: Kalau ceterizine sama ctm mending mana dong? Dirumah ada ceterizine. Dokter: Krn kk menyusui ctm aja ya Pasien: Ada rekomendasi ga dok bedak lotion dan sabun yang bagus? Pasien: Baik dok. Jadi ctm aja ya.. \\\\ \\nDokter: Pokoknya khusus bayi \\\\ \\nPasien: Baik dong... terima kasih dok.. ini sya pake salep ini berapa kalo pakai dok.? Dokter: Klo salep bebas mau dipakai berapa kali mis. habis